In [ ]:
import numpy as np
import matplotlib.pyplot as plt    

Converting ser binary file to numpy array 

In [ ]:
byte_order = 0 # Set the byte order according to the data. 0 for little endian, 1 for big endian.
input_file = 'path/to/your/folder/ser'
# Make sure it's a 1D numpy array of type int32, matching the original data type

if byte_order == 0:
    # If the byte order is little endian, use '<' in dtype
    dtype = '<i4'
elif byte_order == 1:
    # If the byte order is big endian, use '>' in dtype
    dtype = '>i4'

# Converting binary file to a numpy array
with open(input_file, 'rb') as input_serial_file: 
    raw_data = np.frombuffer(input_serial_file.read(), dtype=dtype)

In [ ]:
data_length = len(raw_data)              
num_points_per_fid = 4096                 # If your FIDs are composed of 2048 real and 2048 imaginary points. Total = 4096. Set accordingly.
num_fid = int(data_length/num_points_per_fid)  # Number of FIDs. Match with your data.
print(num_fid)     

In [ ]:
#getting a single numpy array of all the FIDs (256, 2048). Real and imaginary parts are mixed.
fid2d = []

for i  in range(num_fid):
    a = i * num_points_per_fid
    b = (i+1) * num_points_per_fid
    fid_data = raw_data[a:b] 
    reals = fid_data[0::2]    
    imags = fid_data[1::2] 
    fid = reals + 1.j*imags
    fid2d.append(fid)

fid2d = np.array(fid2d) # numpy array of all the FIDs (256, 2048). Real and imaginary parts are mixed.
print(fid2d.shape)

In [ ]:
plt.figure(figsize=(12,8), dpi=100)                   
plt.subplot(2, 1, 1)   
plt.plot(np.real(fid2d[100]), c='r', label='real')  
plt.legend()                                 

plt.subplot(2, 1, 2)                  
plt.plot(np.imag(fid2d[100]),c='k', label='imaginary')
plt.legend()                               

plt.show()   

Converting numpy array to ser binary file

In [ ]:
# Initialize an empty list to store individual FID chunks
fid_chunks = []

# Iterate through each row in fid2d
for fid in fid2d:
    # Separate real and imaginary parts
    reals = np.real(fid)
    imags = np.imag(fid)
    
    # Interleave real and imaginary parts
    fid_chunk = np.column_stack((reals, imags)).flatten()
    
    # Append the fid chunk to the list
    fid_chunks.append(fid_chunk)

# Stack the individual fid chunks to reconstruct the raw_data
raw_data_reconstructed = np.concatenate(fid_chunks)

# Check the shape of the reconstructed raw_data
print("Shape of Reconstructed raw_data:", raw_data_reconstructed.shape)

In [ ]:
re = raw_data - raw_data_reconstructed
print(re, re.shape) # If the difference is zero, then the reconstruction is correct.

In [ ]:
output_file = 'path/to/your/folder/ser'

with open(output_file, 'wb') as output_serial_file: # Conversion of numpy array to ser file. Will be saved at output_file
    output_serial_file.write(np.array(raw_data_reconstructed, dtype=dtype).tobytes())